# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
import sqlite3
import pandas as pd

In [35]:
conn = sqlite3.connect("school.sqlite")
cur = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [61]:
cur.execute("""
                CREATE TABLE contactInfo(
                                        id INTEGER PRIMARY KEY,
                                        FirstName TEXT,
                                        LastName TEXT,
                                        Role TEXT,
                                        TelephoneNumber INTEGER,
                                        Street TEXT,
                                        City TEXT,
                                        State TEXT,
                                        ZipCode TEXT)
        """)

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [62]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [52]:
sql = """   INSERT INTO contactInfo (FirstName, LastName, Role, TelephoneNumber, Street, City, State, ZipCode)
            VALUES(?, ?, ?, ?, ?, ?, ?, ?)
                """

In [63]:
for dictionary in contacts:
    my_values = tuple(list(dictionary.values()))
    cur.execute(sql, my_values)

**Query the Table to Ensure it is populated**

In [64]:
pd.read_sql("""
SELECT *
FROM contactInfo;""", conn)

,id,FirstName,LastName,Role,TelephoneNumber,Street,City,State,ZipCode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,6,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
6,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
7,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [65]:
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [44]:
# Create the grades table
cur.execute("""
                CREATE TABLE grades(
                            userId INTEGER,
                            courseId INTEGER,
                            grade INTEGER)
""")

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [66]:
# Find the duplicate entry
#to remove duplicates, identify criteria to use then use  GROUP BY.
#below we will use Telephone and Zipcode as each is unique for every person

pd.read_sql("""
SELECT *,
    COUNT(*)
FROM contactInfo
GROUP BY TelephoneNumber, ZipCode
HAVING COUNT(*) > 1;
""", conn)


,id,FirstName,LastName,Role,TelephoneNumber,Street,City,State,ZipCode,COUNT(*)
0,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602,2


In [67]:
# Delete the duplicate entry
#to remove duplicates, identify criteria to use then use  GROUP BY.
#below we will use Telephone and Zipcode as each is unique for every person

cur.execute("""
DELETE
    FROM contactInfo
WHERE id NOT IN (
        SELECT MIN(id)
        FROM contactInfo
        GROUP BY TelephoneNumber);
""")

In [68]:
# Check that the duplicate entry was removed
pd.read_sql("""
SELECT *
FROM contactInfo;""", conn)

,id,FirstName,LastName,Role,TelephoneNumber,Street,City,State,ZipCode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,5179695576,3478 Be Sreet,Lansing,MI,48933


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [69]:
# Update Ed's address
cur.execute("""
UPDATE contactInfo
    SET Street = '2910 Simpson Avenue York, PA 17403' WHERE Street = '3478 Be Sreet'

""")


In [19]:
# Query the database to ensure the change was made
pd.read_sql("""
SELECT *
FROM contactInfo;
""", conn)


,id,FirstName,LastName,Role,TelephoneNumber,Street,City,State,ZipCode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,5179695576,"2910 Simpson Avenue York, PA 17403",Lansing,MI,48933


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [70]:
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 